Preuzimanje baze slika i njeno otpakiranje:

In [ ]:
!gdown --id 1vg7_Lw6N_OJ9eoCtHtH5wndKSVjKKplR

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1vg7_Lw6N_OJ9eoCtHtH5wndKSVjKKplR
To: /content/celeb_ds.zip
100% 1.97M/1.97M [00:00<00:00, 150MB/s]


In [ ]:
!unzip -qq celeb_ds.zip

Preuzimanje HCC frontal face kaskade:

In [ ]:
!wget https://raw.githubusercontent.com/opencv/opencv/4.x/data/haarcascades/haarcascade_frontalface_default.xml

--2022-06-02 15:31:16--  https://raw.githubusercontent.com/opencv/opencv/4.x/data/haarcascades/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.009s  

2022-06-02 15:31:16 (102 MB/s) - ‘haarcascade_frontalface_default.xml’ saved [930127/930127]



Importovanje biblioteka:

In [ ]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from copy import deepcopy
from sklearn.neural_network import MLPClassifier

Funkcija za konverziju slike u vektor:

In [ ]:
def convert_image_to_vector(img):
  winSize = (64,64)
  blockSize = (16,16)
  blockStride = (8,8)
  cellSize = (8,8)
  nbins = 9
  derivAperture = 1
  winSigma = 4.
  histogramNormType = 0
  L2HysThreshold = 2.0000000000000001e-01
  gammaCorrection = 0
  nlevels = 64
  hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                          histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
  #compute(img[, winStride[, padding[, locations]]]) -> descriptors
  winStride = (8,8)
  padding = (8,8)
  locations = ((10,20),)
  hist = hog.compute(img,winStride,padding,locations)
  return hist.transpose().tolist()[0]


Funkcija za izdvajanje i vraćanje lica na slici

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def select_face(image, sc=1.3, nn=5):
  image_copy = deepcopy(image)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray, sc, nn)
  
  for (x,y,w,h) in faces:
    roi_color = image_copy[y:y+h, x:x+w]
    return roi_color
  
  return 0

Treniranje sistema sa train data:

In [ ]:
ROOT = "/content/data"

train_dir = ROOT + "/train"
val_dir = ROOT + "/val"

#for directories in os.listdir(train_dir):
 # print(directories)

classes = list(os.listdir(train_dir))
print(classes)

clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(1764, 1000,  500, 200, 100, 50, 1), random_state=5, max_iter=300)

X_train = []
y_train = []

for klasa in classes:  
  current_class = classes[classes.index(klasa)]
  
  for file in os.listdir(train_dir + "/" + current_class):
    #print(file)
    image = cv2.imread(train_dir + "/" + current_class + "/" + file)
    #face=select_face(image)
    #if (isinstance(face, int)==False): vec = convert_image_to_vector(face)
    #else: 
     # vec = convert_image_to_vector(image)

    vec = convert_image_to_vector(image)
    X_train.append(vec)
    y_train.append(classes.index(klasa))


print(len(X_train))
clf.fit(X_train, y_train)

['elton_john', 'jerry_seinfeld', 'madonna', 'ben_afflek']
71


MLPClassifier(alpha=1e-05,
              hidden_layer_sizes=(1764, 1000, 500, 200, 100, 50, 1),
              max_iter=300, random_state=5)

Uvodim nove slike sistemu sa zadatkom da prepozna ličnosti, na čijim je slikama treniran:

In [ ]:
ROOT = "/content/data"

val_dir = ROOT + "/val"

#for directories in os.listdir(train_dir):
 # print(directories)

classes = list(os.listdir(val_dir))
print(classes)

X_val = []
y_val = []

for klasa in classes:  
  current_class = classes[classes.index(klasa)]
  
  for file in os.listdir(val_dir + "/" + current_class):
    #print(file)
    image = cv2.imread(val_dir + "/" + current_class + "/" + file)
    #face=select_face(image)
    #if (isinstance(face, int)==False): vec = convert_image_to_vector(face)
    #else: 
    #  vec = convert_image_to_vector(image)
    vec = convert_image_to_vector(image)

    X_val.append(vec)
    y_val.append(classes.index(klasa))

print(len(X_val))

['elton_john', 'jerry_seinfeld', 'madonna', 'ben_afflek']
20


In [ ]:
y_pred = clf.predict(X_val)

In [ ]:
print(y_pred)
print(y_val)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3]


Ispisujem metričke izvještaje:

In [ ]:
from sklearn.metrics import classification_report


print(classification_report(y_val, y_pred, target_names=classes))

                precision    recall  f1-score   support

    elton_john       0.00      0.00      0.00         5
jerry_seinfeld       0.25      1.00      0.40         5
       madonna       0.00      0.00      0.00         5
    ben_afflek       0.00      0.00      0.00         5

      accuracy                           0.25        20
     macro avg       0.06      0.25      0.10        20
  weighted avg       0.06      0.25      0.10        20



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_pred_train = clf.predict(X_train)

print(classification_report(y_pred_train, y_train, target_names=classes))

                precision    recall  f1-score   support

    elton_john       0.00      0.00      0.00         0
jerry_seinfeld       1.00      0.30      0.46        71
       madonna       0.00      0.00      0.00         0
    ben_afflek       0.00      0.00      0.00         0

      accuracy                           0.30        71
     macro avg       0.25      0.07      0.11        71
  weighted avg       1.00      0.30      0.46        71



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
